### Exercícios da Aula 04
---
 
Author : Halisson S. Gomides - halisson.gomides@gmail.com

Date : 29/01/2021

---

### Conjunto de dados House Sales in King County, USA
>https://www.kaggle.com/harlfoxem/housesalesprediction


### Respondendo perguntas do CEO:

1. Adicione as seguintes informações ao imóvel:
        - O nome da Rua
        - O número do imóvel
        - O nome do Bairro
        - O nome da Cidade
        - O nome do Estado
        
2. Onde tem essas informações: API chamada GEOPY
3. Fazer o link com 'zipcode' e 'lat' e 'long'

In [1]:
import pandas as pd
%config Completer.use_jedi = False 

In [2]:
dfhouses = pd.read_csv('dados/originais/kc_house_data.csv.zip', 
                       parse_dates=['date'], 
                       infer_datetime_format=True,
                      dtype={'condition': str })

In [3]:
dfhouses.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [ ]:
dfhouses.info()

In [ ]:
# Verificando se existem indices duplicados

for k,v in dfhouses['id'].value_counts().to_dict().items():
    if v > 1: 
        print(k, v)
        print(dfhouses.loc[dfhouses['id'] == k])

        

In [6]:
# Removendo as casas com IDs duplicados, mantendo apenas o registro com a data mais recente

df_houses = dfhouses.sort_values(by=['id', 'date'])
# df_order.head(10)
df_houses.drop_duplicates(subset=['id'], keep='last', inplace=True)

# testando o resultado com um id que eu sei que era duplicado
df_houses.query('id == 795000620')

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
17604,795000620,2015-03-11,157000.0,3,1.0,1080,6250,1.0,0,0,...,5,1080,0,1950,0,98168,47.5045,-122.33,1070,6250


#### 1. Qual a quantidade de imóveis por nível?
 - Nivel 0: Preço entre 0.00 e 321.950
 - Nivel 1: Preço entre 321.950 e 450.000
 - Nivel 2: Preço entre 450.000 e 645.000
 - Nivel 3: Preço Acima de 645.000

In [ ]:
# Estabelecendo nivel para as casas de acordo com o preço

df_houses['nivel'] = df_houses['price'].apply(
lambda preco:
    'nivel_0' if 0 <= preco < 321950 else(
        'nivel_1' if 321950 <= preco < 450000 else(
            'nivel_2' if 450000 <= preco < 645000 else 'nivel_3'
        )
    )
)

In [ ]:
df_houses['nivel'].value_counts()

In [ ]:
# !pip install geopy
# !pip install tqdm

In [3]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [4]:
# Inicializando Nominatim API

geolocator = Nominatim( user_agent='geoapiExercicises' )

In [ ]:
# Testando com uma geolocalização aleatoria

response = geolocator.reverse('47.5045,-122.33')
response.raw

#### 2. Adicione as seguintes informações ao imóvel:
 - Nome da Rua
 - Número do imóvel
 - O nome do Bairro
 - O nome da cidade
 - O nome do estado

In [9]:
#  Criando novas colunas vazias no dataframe
import numpy as np

df_houses['road'] = np.nan
df_houses['house_number'] = np.nan
df_houses['neighbourhood'] = np.nan
df_houses['city'] = np.nan
df_houses['state'] = np.nan

In [ ]:
df_houses.tail()

In [ ]:
# verificando a integridade das colunas lat e long

print(df_houses['lat'].isnull().sum())
print(df_houses['long'].isnull().sum())

In [ ]:
# Mostrando barra de progresso 
from tqdm.notebook import trange

for i in trange( len(df_houses) ):
    
    # Implementando cache caso tenha que rodar de novo o laço for usando a coluna "house_number"
    if pd.isna(df_houses.iloc[i, 23]) == False:
        continue
    
    # Logando o percentual de varredura do dataframe    
#     perc = (i/len(df_houses))*100
#     if perc >= 1 and round(perc,2) % 2 == 0:
#         print(f'{perc:.3f}%')
    
    # Montando a string de busca com os dados de 'lat' e 'long'
    query = str(df_houses.iloc[i, 17]) + ',' + str(df_houses.iloc[i, 18])
    
    # API request
    with ThreadPoolExecutor(size) as thp:
        try:
            response = geolocator.reverse(query)
        except GeocoderTimedOut:
            continue
    
    # Popula o dataframe com os dados da API
    if 'road' in response.raw['address']:
        df_houses.iloc[i, 22] = response.raw['address']['road']
    
    if 'house_number' in response.raw['address']:
        df_houses.iloc[i, 23] = response.raw['address']['house_number']
        
    if 'neighbourhood' in response.raw['address']:
        df_houses.iloc[i, 24] = response.raw['address']['neighbourhood']
        
    if 'city' in response.raw['address']:
        df_houses.iloc[i, 25] = response.raw['address']['city']
        
    if 'state' in response.raw['address']:
        df_houses.iloc[i, 26] = response.raw['address']['state']
        

In [7]:
df_houses.loc[0,['lat','long']]

lat     47.5112
long   -122.257
Name: 0, dtype: object

### Agora executando de forma concorrente utilizando multithreading

Conceitos utilizados: 
- https://medium.com/fintechexplained/advanced-python-concurrency-and-parallelism-82e378f26ced
- https://testdriven.io/blog/building-a-concurrent-web-scraper-with-python-and-selenium/

In [10]:
# -----------------------------------------
# Executando de forma concorrente assincronamente
from concurrent.futures import ThreadPoolExecutor
# -----------------------------------------

def consulta_api(lat, long, col):
    
    # Montando a string de busca com os dados de 'lat' e 'long'
    query = str(lat) + ',' + str(long)
    
    try:
        response = geolocator.reverse(query)
    except GeocoderTimedOut:
        pass
        
    if col in response.raw['address']:
        return response.raw['address'][col]
    else:
        return np.nan
    



import tqdm
import multiprocessing
num_processes = multiprocessing.cpu_count()

with ThreadPoolExecutor(num_processes) as thp:
    
    # Popula o dataframe com os dados da API

    df_houses['road'] = list(tqdm.tqdm(thp.map(consulta_api, df_houses['lat'], df_houses['long'], ['road']*len(df_houses), chunksize=10), total=df_houses.shape[0]))
#     df_houses['house_number'] = list(tqdm.tqdm(thp.map(consulta_api, df_houses['lat'], df_houses['long'], 'house_number', chunksize=10), total=df_houses.shape[0]))
#     df_houses['neighbourhood'] = list(tqdm.tqdm(thp.map(consulta_api, df_houses['lat'], df_houses['long'], 'neighbourhood', chunksize=10), total=df_houses.shape[0]))
#     df_houses['city'] = list(tqdm.tqdm(thp.map(consulta_api, df_houses['lat'], df_houses['long'], 'city', chunksize=10), total=df_houses.shape[0]))
#     df_houses['state'] = list(tqdm.tqdm(thp.map(consulta_api, df_houses['lat'], df_houses['long'], 'state', chunksize=10), total=df_houses.shape[0]))
    

  0%|                                                                                        | 0/21436 [00:02<?, ?it/s]


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=47.3262&lon=-122.214&format=json&addressdetails=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000214803FE2E0>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)'))

In [37]:
consulta_api(df_houses.loc[3,'lat'], df_houses.loc[3,'long'], 'road')

'56th Avenue South'

In [ ]:
# salvando num arquivo pra não precisar fazer de novo
df_houses.to_csv('dados/processados/kc_house_data_comp.csv', index=False)

#### 3. Adicione o nível do imóvel no mapa como uma cor

In [1]:
import pandas as pd
import plotly.express as px
#  Importando os componentes necessarios para formatação da tabela de valores
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

%config Completer.use_jedi = False 

In [2]:
df_houses = pd.read_csv('dados/processados/kc_house_data_comp.csv', parse_dates=['date'], 
                       infer_datetime_format=True)

In [ ]:
lst_columns = ['id', 'lat', 'long', 'price', 'nivel']
mapa = px.scatter_mapbox( df_houses[lst_columns], lat='lat', lon='long',  
                     hover_name='id',
                     hover_data=['price'],
                     color_continuous_scale= "IceFire",
                     color='nivel',
                     size='price',
                     height=300,
                     zoom= 9)
mapa.update_layout(mapbox_style='open-street-map')
mapa.update_layout(height=600, margin={'r':0, 't':0, 'l':0, 'b':0})
mapa.show()

#### 4. Adicione o preço do imóvel como o tamanho do ponto no mapa

In [ ]:
lst_columns = ['id', 'lat', 'long', 'price']
mapa = px.scatter_mapbox( df_houses[lst_columns], lat='lat', lon='long',  
                     hover_name='id',
                     hover_data=['price'],
                     color_continuous_scale= px.colors.cyclical.IceFire,
                     size='price',
                     size_max=15,
                     height=300,
                     zoom= 9)
mapa.update_layout(mapbox_style='open-street-map')
mapa.update_layout(height=600, margin={'r':0, 't':0, 'l':0, 'b':0})
mapa.show()

#### 5. Adicione opções de filtros para fazer as próprias análises
  - Eu quero escolher visualizar imóveis com vista para água ou não
  - Eu quero escolher visualizar imóveis até um certo valor de preço
  

In [6]:
import ipywidgets as widgets
from ipywidgets import fixed

In [ ]:
df_houses['is_waterfront'] = df_houses['waterfront'].apply(lambda x: 'yes' if x == 1 else 'no')

style = {'description_width': 'initial'}

# Interative buttons
price_limit = widgets.IntSlider(
    value = 540000,
    min = 75000,
    max = 77000000,
    step = 10,
    description = 'Maximum Price',
    disable = False,
    style = style,
    readout=True
)

waterfront_bar = widgets.Dropdown(
    options = df_houses['is_waterfront'].unique().tolist(),
    value = 'yes',
    description = 'Water View',
    disable=False
)

def update_map(df, waterfront, limit):

#     houses = df.loc[(df['price'] <= limit) & (df['is_waterfront'] == waterfront)][['id', 'lat', 'long', 'price', 'nivel']]
    houses = df.query('price <= @limit & is_waterfront == @waterfront')[['id', 'lat', 'long', 'price', 'nivel']]
    mapa = px.scatter_mapbox( houses, lat='lat', lon='long',  
                     hover_name='id',
                     hover_data=['price'],
                     color_continuous_scale= "IceFire",
                     color='nivel',
                     size='price',
                     height=250,
                     zoom= 9)
    mapa.update_layout(mapbox_style='open-street-map')
    mapa.update_layout(height=600, margin={'r':0, 't':0, 'l':0, 'b':0})
    mapa.show()


In [ ]:
widgets.interactive( update_map, df=fixed(df_houses), waterfront=waterfront_bar, limit=price_limit)

#### 6. Adicionar opções de filtros no últimos dashboard enviado:
   - Visualizar somente valores a partir de uma data disponível para compra

In [ ]:
from matplotlib import gridspec
from matplotlib import pyplot as plt

In [ ]:
# alterando o formato da data

df_houses['year'] = pd.to_datetime(df_houses['date']).dt.isocalendar().year
df_houses['year_week'] = pd.to_datetime(df_houses['date']).dt.strftime('%Y-%U') 
df_houses['date'] = pd.to_datetime(df_houses['date']).dt.strftime('%Y-%m-%d')


# Controle Widget para data
date_limit = widgets.SelectionSlider(
    options= df_houses['date'].sort_values().unique().tolist(),
    value= '2014-12-01',
    description= 'Disponivel',
    continuous_update= False,
    orientation= 'horizontal',
    readout= True
)

def update_map2(data, limit):
    df = data[data['date'] >= limit].copy()
    
    fig = plt.figure( figsize=(21,12) )
    specs = gridspec.GridSpec(ncols=2, nrows=2, figure=fig)
    
    ax1 = fig.add_subplot( specs[0, :] ) # primeira linha, todas colunas
    ax2 = fig.add_subplot( specs[1, 0] ) # segunda linha, primeira coluna
    ax3 = fig.add_subplot( specs[1, 1] ) # segunda linha, segunda coluna
    
    by_year = df[['id', 'year']].groupby('year').sum().reset_index()
    ax1.bar( by_year['year'], by_year['id'] )
    
    by_day = df[['id', 'date']].groupby('date').mean().reset_index()
    ax2.plot( by_day['date'], by_day['id'] )
    ax2.set_title('Avg Price by Day')
    
    by_week = df[['id', 'year_week']].groupby('year_week').mean().reset_index()
    ax3.bar( by_week['year_week'], by_week['id'] )
    ax3.set_title('Avg Price by Week Of Year')
    plt.xticks(rotation=60)
    
    

In [ ]:
widgets.interactive( update_map2, data=fixed(df_houses), limit=date_limit )

### Exercícios:

1. Qual a quantidade de imóveis por nível?
 - Nivel 0: Preço entre 0.00 e R$ 321.950
 - Nivel 1: Preço entre 321.950 e 450.000
 - Nivel 2: Preço entre 450.000 e 645.000
 - Nivel 3: Preço Acima de 645.000

2. Qual a média do tamanho da sala de estar dos imíveis por "size"?
 - Size 0: Tamanho entre 0 e 1427 sqft
 - Size 1: Tamanho entre 1427 e 1910 sqft
 - Size 2: Tamanho entre 1910 e 2550 sqft
 - Size 3: Tamanho acima de 2550 sqtf

3. Adicione as seguintes informações ao conjunto de dados original?
 - Place ID:Identificação da localização
 - OSM type: Open Street Map Type
 - Country: Nome do País
 - Country CODE: Código do País

4. Adicione os seguintes filtros no mapa:
 - Tamanho mínimo da área da sala de estar
 - Número mínino de banheiros
 - Valor máximo do preço
 - Tamanho máximo da área do porão
 - Filtro das condições do imóvel
 - Filtro por ano de construção

5. Adicione os seguintes filtros no Dashboard:

 - Filtro por data disponível para compra
 - Filtro por ano de renovação
 - Filtro se possui vista para água ou não.

In [ ]:
# 1. Qual a quantidade de imóveis por nível?

# df_houses['nivel'].value_counts().plot.bar()

style = {'description_width': 'initial'}
# definindo o controle de nível
level_control = widgets.SelectionSlider(
    options=['Todos'] + df_houses['nivel'].sort_values().unique().tolist(),
    value='Todos',
    description='Nível',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    style= style
)


def bar_graph_contr(df, **controllers_args):

    nivel = controllers_args.get('level')
    if nivel:
        houses = df.query('nivel == @nivel').groupby('nivel').count().reset_index() if nivel != 'Todos' else df.groupby('nivel').count().reset_index()
    
    fig = px.bar(houses, x="nivel", y="id", title='quantidade de imóveis', 
             color_discrete_sequence=px.colors.qualitative.Set1)
    fig.update_layout(
        plot_bgcolor="white",
        margin=dict(t=30,l=0,b=0,r=0),
        bargap=0.8
    )
    fig.show()


    
widgets.interactive( bar_graph_contr, df=fixed(df_houses), level=level_control )

#### 2. Qual a média do tamanho da sala de estar dos imóveis por "size"?
 - Size 0: Tamanho entre 0 e 1427 sqft
 - Size 1: Tamanho entre 1427 e 1910 sqft
 - Size 2: Tamanho entre 1910 e 2550 sqft
 - Size 3: Tamanho acima de 2550 sqtf

In [ ]:
df_houses['size_room'] = df_houses['sqft_living'].apply(
lambda tamanho:
    'size_0' if 0 <= tamanho < 1427 else(
        'size_1' if 1427 <= tamanho < 1910 else(
            'size_2' if 1910 <= tamanho < 2550 else 'size_3'
        )
    )
)

In [ ]:
df_houses['size_room'].sort_values().unique().tolist()

In [ ]:

# filtro de Tamanho
size_control = widgets.SelectionSlider(
    options=['Todos'] + df_houses['size_room'].sort_values().unique().tolist(),
    value='Todos',
    description='Tamanho',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    style= style
)




def dash_graph_contr(df, **controllers_args):

    nivel = controllers_args.get('level')
    size = controllers_args.get('size')
    
    fig = make_subplots(
        rows=1, cols=2,
        shared_xaxes=True,
        vertical_spacing=0.03,
        specs=[[{"type": "bar"},{"type": "bar"}]]
    )
    
    if nivel and size:
        dnivel = df.groupby('nivel').count().reset_index()
        dsize = df.groupby('size_room').agg({'sqft_living': 'mean'}).reset_index()
        if nivel != 'Todos' and size != 'Todos':
            dnivel = df.query('size_room == @size & nivel == @nivel').groupby('nivel').count().reset_index() 
            dsize = df.query('size_room == @size & nivel == @nivel').groupby('size_room').agg({'sqft_living': 'mean'}).reset_index()
        if nivel == 'Todos' and size != 'Todos':
            dnivel = df.query('size_room == @size').groupby('nivel').count().reset_index() 
            dsize = df.query('size_room == @size').groupby('size_room').agg({'sqft_living': 'mean'}).reset_index()
        if nivel != 'Todos' and size == 'Todos':
            dnivel = df.query('nivel == @nivel').groupby('nivel').count().reset_index() 
            dsize = df.query('nivel == @nivel').groupby('size_room').agg({'sqft_living': 'mean'}).reset_index()
    
    
    trace1 = go.Bar(
                name = 'qtd_nivel',
                x = dnivel['nivel'].tolist(),
                y = dnivel['id'].tolist(),
            )
    trace2 = go.Bar(
                name = 'media_tamanho_sala',
                x = dsize['size_room'].tolist(),
                y = dsize['sqft_living'].tolist(),
            )
    
    fig.append_trace(trace1, 1, 1)
    fig.append_trace(trace2, 1, 2)
    fig.update_layout(
        height=600,
        showlegend=False,
        title_text="analise de nivel e tamanho da sala"
    )
    fig.show()


    
widgets.interactive( dash_graph_contr, df=fixed(df_houses), level=level_control, size=size_control )

#### 3. Adicione as seguintes informações ao conjunto de dados original?

- Place ID:Identificação da localização
- OSM type: Open Street Map Type
- Country: Nome do País
- Country CODE: Código do País

In [ ]:
# Testando com uma geolocalização aleatoria

response = geolocator.reverse('47.5045,-122.33')
response.raw

In [ ]:
#  Criando novas colunas vazias no dataframe
from numpy import nan

df_houses['place_id'] = nan
df_houses['osm_type'] = nan
df_houses['country'] = nan
df_houses['country_code'] = nan


In [ ]:
df_houses.columns.size
df_houses.columns.get_loc('long')

In [2]:
def iterdict(d, chave):
    '''
    Função para iterar recursivamente sobre um dicionario e retornar o valor da chave passada como parametro
    
    Keyword arguments:
    d -- dictionary
    chave -- name of the key to be searched
    '''
    from numpy import nan
    
    if chave in d: 
        return d[chave]
    for v in d.values():        
        if isinstance(v, dict):
            return iterdict(dict(v), chave)
            
    return  nan



def get_geo_attr(key_tuple, df ):
    '''
    Função para adicionar ao dataframe informações da API geopy.geocoders
    
    Keyword arguments:
    key_tuple -- tupla com os nomes das chaves que contém as informações da API a serem adicionadas ao dataframe
    df -- dataframe que deve conter as colunas a serem populadas com os nomes das chaves key_tuple, bem como as 
    colunas "lat" e "long" com dados de latitude e longitude a serem usados como query para pesquisa na API
    '''
    
    from geopy.geocoders import Nominatim
    from geopy.exc import GeocoderTimedOut
    from tqdm.notebook import trange
    
    geolocator = Nominatim( user_agent='geoapiExercicises' )

    for i in trange( len(df) ):

        # Implementando cache caso tenha que rodar de novo o laço for. Verifica o preenchimento na coluna cujo nome 
        # é a primeteamira chave da tupla key_tuple
        if pd.isna(df.iloc[i, df.columns.get_loc(key_tuple[0])]) == False:
            continue

        # Montando a string de busca com os dados de 'lat' e 'long'
        query = str(df.iloc[i, df.columns.get_loc('lat')]) + ',' + str(df.iloc[i, df.columns.get_loc('long')])

        # API request
        try:
            response = geolocator.reverse(query)
        except GeocoderTimedOut:
            continue

        # Popula o dataframe com os dados da API conforme as chaves passadas na tupla key_tuple:
        for chave in key_tuple:
            df.iloc[i, df.columns.get_loc(chave)] = iterdict(dict(response.raw), chave)
           
    
    return df


In [ ]:
novas_infos = ('place_id', 'osm_type', 'country', 'country_code')
df_houses = get_geo_attr(novas_infos, df_houses)

In [ ]:
# verificando se as informações preenchidas
df_houses.sample(10)

In [ ]:
# salvando num arquivo pra não precisar fazer de novo
df_houses.to_csv('dados/processados/kc_house_data_comp.csv', index=False)

#### 4. Adicione os seguintes filtros no mapa:
 - Tamanho mínimo da área da sala de estar
 - Número mínino de banheiros
 - Valor máximo do preço
 - Tamanho máximo da área do porão
 - Filtro das condições do imóvel
 - Filtro por ano de construção

In [4]:
# Abrindo o arquivo já com os dados que foram coletados da API Geopy
# 
# Isso foi feito, pois ao fechar o notebook antes de concluir todos o exercícios, e depois abrir novamente
# para dar continuidade aos exercícios, não é necessário coletar novamente os dados da API, pois é muito demorado
# 

df_houses = pd.read_csv('dados/processados/kc_house_data_comp.csv', parse_dates=['date'], 
                       infer_datetime_format=True, dtype={'condition': str })

In [7]:
from ipywidgets import Layout

style = {'description_width': 'initial', 'handle_color': 'yellow'}
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between',
    width='auto'
)

# Filtro de tamanho da área da sala de estar
livroom_limit = widgets.IntSlider(
    value = 50,
    min = 5,
    max = 14000,
    step = 2,
    description = 'T. Min. Sala',
    disable = False,
    style = style,
    readout=True,
    layout = form_item_layout
)

# Filtro de Número mínino de banheiros
bath_limit = widgets.IntSlider(
    value = 0,
    min = 1,
    max = 10,
    step = 1,
    description = 'Qtd. Min. Banheiros',
    disable = False,
    style = style,
    readout=True,
    layout = form_item_layout
)

# Filtro de Valor máximo do preço
price_limit = widgets.IntSlider(
    value = 540000,
    min = 75000,
    max = 77000000,
    step = 10,
    description = 'Max. Preço',
    disable = False,
    style = style,
    readout=True,
    layout = form_item_layout
)

# Filtro das condições do imóvel
condition_limit = widgets.SelectionSlider(
    options=['Todas'] + df_houses['condition'].sort_values().unique().tolist(),
    value='Todas',
    description='Condição',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    style= style,
    layout = form_item_layout
)

# Filtro por ano de construção
yb = [(i,i) for i in df_houses['yr_built'].sort_values().unique()]
yrbuilt_limit = widgets.SelectionRangeSlider(
    options=yb,
    index=(0,len(yb)-1),
    description='Ano Constr.',
    disabled=False,
    style = style,
    readout=True,
    layout = form_item_layout
)


In [13]:
def map_contr(df, **controllers_args):

    livroom = controllers_args.get('livroom')
    bath = controllers_args.get('bath')
    price = controllers_args.get('price')
    condition = controllers_args.get('condition')
    yrbuilt = controllers_args.get('yrbuilt')
    
    # Exercicio 05
    date = controllers_args.get('date')
    yrreno = controllers_args.get('yrreno')
    waterfront = controllers_args.get('waterfront')
    
    
    dhouses = df.query('sqft_living >= @livroom & bathrooms >= @bath & price <= @price & @yrbuilt[0] <= yr_built <= @yrbuilt[1]')[['id', 'lat', 'long', 'price', 'sqft_living']]
    if condition != 'Todas':
        dhouses = df.query('sqft_living >= @livroom & bathrooms >= @bath & price <= @price & @yrbuilt[0] <= yr_built <= @yrbuilt[1] & condition == @condition')[['id', 'lat', 'long', 'price', 'sqft_living']]

    if date and yrreno and waterfront:
        if condition != 'Todas':
            dhouses = df.query('sqft_living >= @livroom & bathrooms >= @bath & price <= @price & @yrbuilt[0] <= yr_built <= @yrbuilt[1] & condition == @condition & \
                           @date[0] <= date <= @date[1] & @yrreno[0] <= yr_renovated <= @yrreno[1] & waterfront == @waterfront')[['id', 'lat', 'long', 'price', 'sqft_living']]
        else:
            dhouses = df.query('sqft_living >= @livroom & bathrooms >= @bath & price <= @price & @yrbuilt[0] <= yr_built <= @yrbuilt[1] & \
                           @date[0] <= date <= @date[1] & @yrreno[0] <= yr_renovated <= @yrreno[1] & waterfront == @waterfront')[['id', 'lat', 'long', 'price', 'sqft_living']]
    
    
    
    mapa = px.scatter_mapbox( dhouses, lat='lat', lon='long',  
                     hover_name='id',
                     hover_data=['price'],
                     color_continuous_scale= px.colors.diverging.Portland,
                     color='sqft_living',
                     size='price',
                     height=250,
                     zoom= 9)
    mapa.update_layout(mapbox_style='open-street-map')
    mapa.update_layout(height=600, margin={'r':0, 't':0, 'l':0, 'b':0})
    mapa.show()


    
widgets.interactive( map_contr, df=fixed(df_houses), 
                    livroom=livroom_limit, 
                    bath=bath_limit, 
                    price=price_limit, 
                    condition=condition_limit, 
                    yrbuilt=yrbuilt_limit )

interactive(children=(IntSlider(value=50, description='T. Min. Sala', layout=Layout(display='flex', flex_flow=…

#### 5. Adicione os seguintes filtros no Dashboard:

 - Filtro por data disponível para compra
 - Filtro por ano de renovação
 - Filtro se possui vista para água ou não.

In [14]:
#  Filtro por data disponível para compra
dates = [(pd.to_datetime(i).strftime('%d/%m/%Y'),pd.to_datetime(i)) for i in df_houses['date'].sort_values().unique()]
date_limit = widgets.SelectionRangeSlider(
    options=dates,
    index=(0,len(dates)-1),
    description='Data',
    disabled=False,
    style = style,
    readout=True,
    layout = form_item_layout
)

# Filtro por ano de renovação
yr = [(i,i) for i in df_houses['yr_renovated'].sort_values().unique()]
yrreno_limit = widgets.SelectionRangeSlider(
    options=yr,
    index=(0,len(yr)-1),
    description='Ano Renov.',
    disabled=False,
    style = style,
    readout=True,
    layout = form_item_layout
)


# Filtro se possui vista para águas ou não.
waterfront_bar = widgets.Dropdown(
    options = df_houses['waterfront'].unique().tolist(),
    value = 0,
    description = 'Vista para água',
    disable=False,
    style = style,
    layout = form_item_layout
)

In [15]:
widgets.interactive( map_contr, df=fixed(df_houses), 
                    livroom=livroom_limit, 
                    bath=bath_limit, 
                    price=price_limit, 
                    condition=condition_limit, 
                    yrbuilt=yrbuilt_limit,
                    date = date_limit,
                    yrreno = yrreno_limit,
                    waterfront = waterfront_bar)

interactive(children=(IntSlider(value=50, description='T. Min. Sala', layout=Layout(display='flex', flex_flow=…

In [5]:
df_houses['date'].value_counts()

2014-06-23    142
2014-06-25    130
2014-06-26    127
2015-04-27    126
2015-03-25    123
             ... 
2014-07-27      1
2014-08-03      1
2014-11-15      1
2014-11-02      1
2015-05-24      1
Name: date, Length: 372, dtype: int64